<a href="https://colab.research.google.com/github/Zeyu-Li/RL-Now/blob/main/CartPole/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RL $\to$ CartPole

Following [Deep Reinforcement Learning Tutorial from Nicholas Renotte](https://www.youtube.com/watch?v=cO5g5qLrLSo)

In [1]:
# dependencies
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Test the gym
import gym
import random

In [3]:
# get env set up
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

/usr/local/lib/python3.8/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
episodes = 10

for episode in range(episodes):
  state = env.reset()
  finish = False
  score = 0
  while not finish:
    # show simulation
    # env.render()
    # randomly pick an action
    action = random.choice([0,1])
    n_state, reward, finish, info = env.step(action)
    score += reward

  print(f'Episode: {episode + 1} Score {score}')


Episode: 1 Score 21.0
Episode: 2 Score 12.0
Episode: 3 Score 17.0
Episode: 4 Score 14.0
Episode: 5 Score 22.0
Episode: 6 Score 19.0
Episode: 7 Score 22.0
Episode: 8 Score 26.0
Episode: 9 Score 13.0
Episode: 10 Score 12.0


In [5]:
# Use Keras
import numpy as np
# from tensorflow.keras.models import Sequential
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/iterator_ops.py:546: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  class IteratorBase(collections.Iterator, trackable.Trackable,
/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/dataset_ops.py:106: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  class DatasetV2(collections.Iterable, tracking_base.Trackable,


In [14]:
def build_model(states, actions):
  model = Sequential()
  # 1 input and 2 layers of dense nodes
  model.add(Flatten(input_shape=(1, states)))
  model.add(Dense(24, activation='relu'))
  model.add(Dense(24, activation='relu'))
  model.add(Dense(actions, activation='linear'))

  return model


In [15]:
model = build_model(states, actions)

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [17]:
# create keras agent
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [18]:
def build_agent(model, actions):
  policy = BoltzmannQPolicy()
  # create an agent with the right params
  memory = SequentialMemory(limit=50000, window_length=1)
  dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
  return dqn


In [19]:
# instanciate, compile, then train the model
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training_utils.py:1446: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(sample_weight_mode, collections.Mapping):
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 8:42 - reward: 1.0000

/usr/local/lib/python3.8/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   31/10000 [..............................] - ETA: 3:20 - reward: 1.0000

/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch_idxs

10000/10000 [==============================] - 82s 8ms/step - reward: 1.0000
112 episodes - episode_reward: 87.696 [8.000, 200.000] - loss: 2.261 - mae: 18.217 - mean_q: 36.938

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 76s 8ms/step - reward: 1.0000
57 episodes - episode_reward: 176.702 [137.000, 200.000] - loss: 3.480 - mae: 37.121 - mean_q: 75.120

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 99s 10ms/step - reward: 1.0000
56 episodes - episode_reward: 179.500 [129.000, 200.000] - loss: 2.829 - mae: 38.560 - mean_q: 77.785

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 101s 10ms/step - reward: 1.0000
54 episodes - episode_reward: 184.741 [115.000, 200.000] - loss: 3.915 - mae: 37.012 - mean_q: 74.505

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 90s 9ms/step - reward: 1.0000
done, took 448.427 seconds


In [20]:
scores = dqn.test(env, nb_episodes=50, visualize=False)
score_mean = np.mean(scores.history['episode_reward'])
print(f'The mean score is: {score_mean}')

Testing for 50 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 2

In [21]:
# saving weights
dqn.save_weights('dqn_weights.h5f', overwrite=True)

## reload
# dqn.load_weights('dqn_weights.h5f')
